In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set(style="whitegrid")

In [2]:
# # Load Data
# train_df = pd.read_csv("../data/train.csv")
# train_df.Label = train_df.Label.astype('category')

# test_df = pd.read_csv("../data/test.csv")
# validation_df = pd.read_csv("../data/valid.csv")

In [3]:
# train_df.describe()

In [4]:
# pd.options.display.max_colwidth=500
# train_df.head()

In [5]:
# train_df.Label.hist()

In [6]:
# train_df_context_len = train_df.Context.str.split(" ").apply(len)
# train_df_Utterance_len = train_df.Utterance.str.split(" ").apply(len)

In [7]:
# print(train_df_context_len.describe()['max'])
# print(train_df_Utterance_len.describe())

In [8]:
# pd.options.display.max_colwidth=50
# test_df.head(1)
# test_df.columns.values.tolist()

In [9]:
# test_df_context_len = test_df['Context'].str.split(" ").apply(len)
# print(test_df_context_len.describe()['max'])

# for key in test_df.columns.values.tolist():
#     a = test_df[key].str.split(" ").apply(len)
#     print(a.describe()['max'])



# test_df_context_len = train_df.Distractor_0.str.split(" ").apply(len)

In [10]:
# from keras.preprocessing.text import Tokenizer
# tokenizer = Tokenizer()
# train_file = open("../data/train.csv",'r')
# tokenizer.fit_on_texts(train_file)


In [11]:
# test_file = open("../data/test.csv",'r')
# valid_file = open("../data/valid.csv",'r')
# tokenizer.fit_on_texts(test_file)
# tokenizer.fit_on_texts(valid_file)

In [12]:
# print(len(tokenizer.word_index))
# with tf.Session() as sess:
# #     a = tokenizer.texts_to_sequences(
# #         tf.string_split(("hello world"," ")).values)
#     a = tf.string_split(("hello world"," ")).values
#     print(sess.run(a))

In [13]:
# with open("/mnt/cephfs/dataset/sentence_with_punc/ch_en_15k.dict",'r') as f:
#     print(f.readlines()[0:10])

# with open("./tokenizer.dict",'w') as f:
#     count = 0
#     f.write('<unk>'+" "+str(0)+'\n')
#     for word,i in tokenizer.word_index.items():
#         count += 1
#         f.write(word+" "+str(i)+'\n')
#         if count%1000 == 0:
#             print(count/434054*100.0)
        
# with open("./tokenizer.dict",'r') as f:

#     print(f.readlines()[0:10])

In [2]:
import tensorflow as tf

class reader(object):
    def __init__(self):
        self.train_file_path  = "../data/train.csv"
        self.dict_file_path = "./tokenizer.dict"
        self._CSV_COLUMN_DEFAULTS = [[''],[''],[1]]
        self._CSV_CLOLUMNS = ['Context','Utterance','Label']
        self.word_dict = {}
        with open(self.dict_file_path, 'r') as f:
            for l in f.readlines():
                word, idx = l.split(' ')[0], l.split(' ')[1]
                self.word_dict[word] = float(idx)
        print("word_dict size:", len(self.word_dict))
        self.UNK_ID = self.word_dict['<unk>']
        
        
        kv_initializer = tf.contrib.lookup.TextFileInitializer(
            self.dict_file_path,tf.string,0,tf.float32,1,delimiter=" ")
        self.lookup_table = tf.contrib.lookup.HashTable(kv_initializer,0)
        dataset = tf.data.TextLineDataset(self.train_file_path).repeat()
        dataset = dataset.skip(1)
        dataset = dataset.map(self.parseCSVLine)
        dataset = dataset.map(self.lookUpDict)
        
        
        dataset = dataset.filter(lambda line: line['lenQuery']<160)
        dataset = dataset.filter(lambda line: line['lenResponse']<160)
#         filter(lambda line: tf.not_equal(tf.substr(line, 0, 1), "#")
        dataset = dataset.padded_batch(512,padded_shapes={'Context':[160],
                                                         'Utterance':[160],
                                                         'Label':[1],
                                                         'lenQuery':[],
                                                         'lenResponse':[]})
#         dataset = dataset.batch(10)
        self.iterator = dataset.make_initializable_iterator()
        
    def parseCSVLine(self,value):
        columns = tf.decode_csv(value,self._CSV_COLUMN_DEFAULTS)
        fetures = dict(zip(self._CSV_CLOLUMNS,columns))
        return fetures

    def lookUpDict(self,value):
#         print(value['Query'])
#         value['Query'] = self.lookup_table.lookup(value['Context'])

        value['Context'] =  self.lookup_table.lookup(
            tf.string_split((value['Context']," "))).values
        value['lenQuery'] = tf.size(value['Context'])
        value['Utterance'] =  self.lookup_table.lookup(
            tf.string_split((value['Utterance']," "))).values
        value['lenResponse'] = tf.size(value['Utterance'])
        value['Label'] = [tf.cast(value['Label'],tf.float32)]
        return value
    
    def init_reader(self,sess):
        sess.run(self.lookup_table.init)
    
    def epoch_input(self):
        return self.iterator

# with tf.Session() as sess:
#     train_textline = reader()
#     train_textline.init_reader(sess)
#     sess.run(train_textline.epoch_input().initializer)
#     a = sess.run(train_textline.epoch_input().get_next()['Label'])
#     for i in range(2):
#         print(a)

In [105]:

class test_reader(object):
    def __init__(self):
        self.test_file_path  = "../data/test.csv"
        self.dict_file_path = "./tokenizer.dict"
        self._CSV_COLUMN_DEFAULTS = [[''],[''],[''],[''],['']
                                    ,[''],[''],[''],[''],[''],['']]
        self._CSV_CLOLUMNS = ['Context','GroundTruth','D0'
                             ,'D1','D2','D3','D4','D5','D6'
                             ,'D7','D8']
        self.word_dict = {}
        
        with open(self.dict_file_path, 'r') as f:
            for l in f.readlines():
                word, idx = l.split(' ')[0], l.split(' ')[1]
                self.word_dict[word] = int(idx)
        print("word_dict size:", len(self.word_dict))
        self.UNK_ID = self.word_dict['<unk>']
        
        kv_initializer = tf.contrib.lookup.TextFileInitializer(
            self.dict_file_path,tf.string,0,tf.float32,1,delimiter=" ")
        self.lookup_table = tf.contrib.lookup.HashTable(kv_initializer,self.UNK_ID)
        dataset = tf.data.TextLineDataset(self.test_file_path)
        dataset = dataset.skip(1)
        dataset = dataset.repeat()
        dataset = dataset.map(self.parseCSVLine)
        dataset = dataset.map(self.lookUpDict)

        for key in self._CSV_CLOLUMNS:
            dataset = dataset.filter(lambda line: tf.size(line[key])<150)
        padded_shapes = {}
        for key in self._CSV_CLOLUMNS:
            padded_shapes[key]=[150]
        dataset = dataset.padded_batch(512,padded_shapes=padded_shapes)
        dataset =  dataset.map(self.get2together)
        self.iterator = dataset.make_initializable_iterator()
    
    def get2together(self, value):
#         value['Compare'] = []
#         for key in self._CSV_CLOLUMNS:
#             if key!='Context':
#                 value['Compare'].append([value['Context'],value[key]])
#         for key in self._CSV_CLOLUMNS:
#             value.pop(key)
#         return value
        
#         value['Compare'] = []
#         for i in range(1):
#             value['Compare'].append([value['Context'][i],value['GroundTruth'][i]])
#         for key in self._CSV_CLOLUMNS:
#             value.pop(key)
        return value



    
    def parseCSVLine(self,value):
        columns = tf.decode_csv(value,self._CSV_COLUMN_DEFAULTS)
        fetures = dict(zip(self._CSV_CLOLUMNS,columns))
        return fetures

    def lookUpDict(self,value):
        for key in value:
            value[key] = self.lookup_table.lookup(
                tf.string_split((value[key]," "))).values

        return value
    
    def init_reader(self,sess):
        sess.run(self.lookup_table.init)
    
    def epoch_input(self):
        return self.iterator
    
# with tf.Session() as sess:
#     test_textline = test_reader()
#     test_textline.init_reader(sess)
#     sess.run(test_textline.epoch_input().initializer)
#     for i in range(1):
#         print(sess.run(test_textline.epoch_input().get_next()['GroundTruth']))


In [4]:
#embedding
import numpy as np
embeddings_index = {}
embedding_file = "./glove.6B.100d.txt"
with open(embedding_file, 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except ValueError:
            continue
        embeddings_index[word] = coefs

word_dict = {}
dict_file_path = "./tokenizer.dict"      
with open(dict_file_path, 'r') as f:
    for l in f.readlines():
        word, idx = l.split(' ')[0], l.split(' ')[1]
        word_dict[word] = int(idx)
print("word_dict size:", len(word_dict))

MAX_NB_WORDS = len(embeddings_index)
num_words = min(MAX_NB_WORDS, len(word_dict)) + 1
embedding_matrix = np.zeros((num_words , 100))

for word, i in word_dict.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

word_dict size: 434055


Using TensorFlow backend.


word_dict size: 434055
word_dict size: 434055
word_dict size: 434055


In [8]:
import keras
import numpy as np
import tensorflow as tf
from keras.layers import Dense, Bidirectional
from keras.layers import Embedding, LSTM, Input
from keras.models import Sequential
from keras.models import Model
from keras.optimizers import SGD
from keras import backend as K
from keras.layers import merge
from keras.utils import multi_gpu_model


train_xc_textline = reader()
train_xr_textline = reader()
train_xl_textline = reader()
train_xc_textline.init_reader(K.get_session())
train_xr_textline.init_reader(K.get_session())
train_xl_textline.init_reader(K.get_session())
K.get_session().run(train_xc_textline.epoch_input().initializer)
K.get_session().run(train_xr_textline.epoch_input().initializer)
K.get_session().run(train_xl_textline.epoch_input().initializer)
train_c = train_xc_textline.epoch_input().get_next()['Context']
train_r = train_xr_textline.epoch_input().get_next()['Utterance']
train_l = train_xl_textline.epoch_input().get_next()['Label']
encoder = Sequential()
embeddin_layer = Embedding(output_dim=100,
                        input_dim=MAX_NB_WORDS+1,
                        input_length=160,
                        weights=[embedding_matrix],
                        mask_zero=True,
                        trainable=True)
lstm_layer = LSTM(units=200)
encoder.add(embeddin_layer)
encoder.add(lstm_layer)

context_input = Input(shape=(160,),tensor=train_c)
response_input = Input(shape=(160,),tensor=train_r)
# context_input = Input(shape=(160,), dtype='float32')
# response_input = Input(shape=(160,), dtype='float32')

context_branch = encoder(context_input)
response_branch = encoder(response_input)

concatenated = keras.layers.Dot(axes=1)([context_branch, response_branch])
out = Dense((1), activation = "sigmoid") (concatenated)
dual_encoder = Model([context_input, response_input], out)
# dual_encoder = multi_gpu_model(dual_encoder,gpus=1)
sgd = SGD(lr=0.1,momentum=0.1,decay=0.1,nesterov=False)
dual_encoder.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'],
                target_tensors = [train_l])
dual_encoder.summary()
dual_encoder.fit(epochs=1,steps_per_epoch=1000)


word_dict size: 434055
word_dict size: 434055
word_dict size: 434055
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 160)          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 160)          0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 200)          40240900    input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
dot_3 (Dot)                     (None, 1

In [29]:
dual_encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 160)          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 160)          0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 200)          40240900    input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
dot_3 (Dot)                     (None, 1)            0           sequential_3[1][0]               
          

In [9]:
weight_path = './saved_wt.h5'
dual_encoder.save_weights(weight_path)


In [44]:
K.clear_session()
test_xc_textline = test_reader()
test_xr_textline = test_reader()
test_xc_textline.init_reader(K.get_session())
test_xr_textline.init_reader(K.get_session())
K.get_session().run(test_xc_textline.epoch_input().initializer)
K.get_session().run(test_xr_textline.epoch_input().initializer)
test_c = test_xc_textline.epoch_input().get_next()['Context']
test_r = test_xr_textline.epoch_input().get_next()['GroundTruth']


train_xc_textline = reader()
train_xr_textline = reader()
train_xl_textline = reader()
train_xc_textline.init_reader(K.get_session())
train_xr_textline.init_reader(K.get_session())
train_xl_textline.init_reader(K.get_session())
K.get_session().run(train_xc_textline.epoch_input().initializer)
K.get_session().run(train_xr_textline.epoch_input().initializer)
K.get_session().run(train_xl_textline.epoch_input().initializer)
train_c = train_xc_textline.epoch_input().get_next()['Context']
train_r = train_xr_textline.epoch_input().get_next()['Utterance']
train_l = train_xl_textline.epoch_input().get_next()['Label']




word_dict size: 434055
word_dict size: 434055
word_dict size: 434055
word_dict size: 434055
word_dict size: 434055


In [106]:
def test_model(key):
    K.clear_session()
    test_xc_textline = test_reader()
    test_xr_textline = test_reader()
    test_xc_textline.init_reader(K.get_session())
    test_xr_textline.init_reader(K.get_session())
    K.get_session().run(test_xc_textline.epoch_input().initializer)
    K.get_session().run(test_xr_textline.epoch_input().initializer)
    test_c = test_xc_textline.epoch_input().get_next()['Context']
    test_r = test_xr_textline.epoch_input().get_next()[key]
    
    
    test_encoder = Sequential()
    embeddin_layer = Embedding(output_dim=100,
                            input_dim=MAX_NB_WORDS+1,
                            input_length=160,
                            weights=[embedding_matrix],
                            mask_zero=True,
                            trainable=True)
    lstm_layer = LSTM(units=200)
    test_encoder.add(embeddin_layer)
    test_encoder.add(lstm_layer)

    test_context_input = Input(shape=(160,),tensor=test_c)
    test_response_input = Input(shape=(160,),tensor=test_r)
    # context_input = Input(shape=(160,), dtype='float32')
    # response_input = Input(shape=(160,), dtype='float32')

    test_context_branch = test_encoder(test_context_input)
    test_response_branch = test_encoder(test_response_input)

    test_concatenated = keras.layers.Dot(axes=1)(
                                [test_context_branch, 
                                 test_response_branch])
    test_out = Dense((1), activation = "sigmoid") (test_concatenated)
    test_dual_encoder = Model([test_context_input, test_response_input], test_out)
    test_dual_encoder.load_weights(weight_path)
    test_dual_encoder.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])
#     test_dual_encoder.summary()

    his = test_dual_encoder.predict(x=None,steps=100)
    return his


In [107]:
key_list =  ['GroundTruth','D0','D1','D2','D3','D4','D5','D6','D7','D8']
res = []
for key in key_list:
    print(key)
    res.append(test_model(key))

GroundTruth
word_dict size: 434055
word_dict size: 434055
D0
word_dict size: 434055
word_dict size: 434055
D1
word_dict size: 434055
word_dict size: 434055
D2
word_dict size: 434055
word_dict size: 434055
D3
word_dict size: 434055
word_dict size: 434055
D4
word_dict size: 434055
word_dict size: 434055
D5
word_dict size: 434055
word_dict size: 434055
D6
word_dict size: 434055
word_dict size: 434055
D7
word_dict size: 434055
word_dict size: 434055
D8
word_dict size: 434055
word_dict size: 434055


In [114]:
res = np.array(res)
count = 0
for i in range(len(res[0])):
    if 0 in res[:,i].reshape(10).argsort()[-2:]:
        count+=1
print(count/(512*100))

0.51234375


In [113]:
print(res[:,0].reshape(10))
res[:,0].reshape(10)[-1:]

[0.7787639  0.812789   0.21565938 0.42614526 0.23397344 0.7867426
 0.58415186 0.5378984  0.39631486 0.39908376]


array([0.39908376], dtype=float32)

In [1]:
a ={}
a['bb'] = 1
a['cc'] = 2


In [2]:
c,d = a

In [3]:
print(C)

NameError: name 'C' is not defined